# Amazon HealthLake Imaging Integrated with MONAI Deploy App SDK 

In the following sections, we will demonstrate how to create a MONAI Deploy application package using the MONAI Deploy App SDK, which will read data from Amazon HealthLake Imaging

## Creating Operators and connecting them in Application class

We will implement an application that consists of the new DataLoader Operators:

- **AHLIDataLoaderOperator**:
    - **Input(json_file)**: a file with Amazon HealthLake Imaging DatastoreId and ImageSetIds to be loaded ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])



This MAP is derived from this tutorial: https://github.com/Project-MONAI/monai-deploy-app-sdk/blob/main/notebooks/tutorials/03_segmentation_app.ipynb

### Setup environment


In [9]:
%%sh
pip install --upgrade pip
pip install --upgrade boto3 botocore
pip install -q "torch>=1.10.2" "numpy>=1.21" "nibabel>=3.2.1" "pydicom>=1.4.2" "highdicom>=0.18.2" "SimpleITK>=2.0.0" "typeguard>=2.12.1" "itk>=5.3rc4" "itkwidgets[all]>=1.0a23"
pip install -q tqdm pathlib2 pylibjpeg-openjpeg 
pip install --upgrade -q "monai" "monai-deploy-app-sdk" 
pip install --upgrade -q awscliv2 AHItoDICOMInterface

In [4]:
%store -r

### Setup imports

Let's import necessary classes/decorators to define Application and Operator.

In [5]:
import logging
import os
from os import path
from numpy import uint8
import boto3
import json
import monai.deploy.core as md
from pydicom.sr.codedict import codes
from monai.deploy.core import Application, resource, Image, IOType
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import (
    BundleConfigNames,
    IOMapping,
    MonaiBundleInferenceOperator,
)
logging.basicConfig( level=logging.INFO )
logging.getLogger('AHItoDICOMInterface').setLevel(logging.CRITICAL)

# from src.code.Api import MedicalImaging
# medicalimaging = MedicalImaging()
from src.code.ahi_data_loader_operator import AHLIDataLoaderOperator
from AHItoDICOMInterface.AHItoDICOM import AHItoDICOM
helper = AHItoDICOM()
instances = helper.DICOMizeImageSet(datastore_id=datastoreId , image_set_id=next(iter(imageSetIds)))

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
Invalid value for VR IS: 'F'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.


### Creating Application class

Our application class would look like below.

It defines `App` class, inheriting [Application](/modules/_autosummary/monai.deploy.core.Application) class.

The requirements (resource and package dependency) for the App can be specified by using [@resource](/modules/_autosummary/monai.deploy.core.resource) and [@env](/modules/_autosummary/monai.deploy.core.env) decorators.

The base class method, `compose`, is overridden. Objects required for DICOM parsing, series selection (selecting the first series for the current release), pixel data conversion to volume image, and segmentation instance creation are created, so is the model-specific `SpleenSegOperator`. The execution pipeline, as a Directed Acyclic Graph, is created by connecting these objects through <a href="../../modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>.

In [6]:
@resource(cpu=1, gpu=1, memory="7Gi")
class AISpleenSegApp(Application):
    def __init__(self, ahi_client, *args, **kwargs):
        """Creates an application instance."""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        self.ahi_client = ahi_client
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.info(f"Begin {self.run.__name__}")
        try:
            super().run(*args, **kwargs)
        except:
            self._logger.info("errrrror")
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""
        logging.info(f"Begin {self.compose.__name__}")

        # Create the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = AHLIDataLoaderOperator(self.ahi_client)
        series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name is optional when the app has only one model.
        # The bundle_path argument optionally can be set to an accessible bundle file path in the dev
        # environment, so when the app is packaged into a MAP, the operator can complete the bundle parsing
        # during init to provide the optional packages info, parsed from the bundle, to the packager
        # for it to install the packages in the MAP docker image.
        # Setting output IOType to DISK only works only for leaf operators, not the case in this example.
        #
        # Pertinent MONAI Bundle:
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation

        config_names = BundleConfigNames(config_names=["inference"])  # Same as the default

        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            bundle_config_names=config_names,
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        segment_descriptions = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.1.0",
            )
        ]

        custom_tags = {"SeriesDescription": "AI generated Seg, not for clinical use."}

        dicom_seg_writer = DICOMSegmentationWriterOperator(
            segment_descriptions=segment_descriptions, custom_tags=custom_tags
        )

        # Create the processing pipeline, by specifying the source and destination operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {"image": "image"})
        # Note below the dicom_seg_writer requires two inputs, each coming from a source operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer, {"pred": "seg_image"})
        # Create the surface mesh STL conversion operator and add it to the app execution flow, if needed, by
        # uncommenting the following couple lines.
        # stl_conversion_op = STLConversionOperator(output_file="stl/spleen.stl")
        # self.add_flow(bundle_spleen_seg_op, stl_conversion_op, {"pred": "image"})

        logging.info(f"End {self.compose.__name__}")

# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
# For list of string values, e.g. "ImageType": ["PRIMARY", "ORIGINAL"], it is a match if all elements
# are all in the multi-value attribute of the DICOM series.

Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "Modality": "(?i)CT"
            }
        }
    ]
}
"""


## Executing app locally

We can execute the app in the Jupyter notebook. Note that the DICOM files of the CT Abdomen series must be present in the `dcm` and the Torch Script model at `model.ts`. Please use the actual path in your environment.


In [7]:
with open('inputImageSets.json', 'w') as f:
    f.write(json.dumps({
        "datastoreId": datastoreId, 
        "imageSetId": next(iter(imageSetIds))
    }))

app = AISpleenSegApp(helper)

app.run(input="inputImageSets.json", output="output", model="src/model.ts")

INFO:root:Begin compose
INFO:root:End compose
INFO:__main__.AISpleenSegApp:Begin run


Going to initiate execution of operator AHLIDataLoaderOperator
Executing operator AHLIDataLoaderOperator (Process ID: 30, Operator ID: ca820b9f-be71-482b-aed7-41e9cc43b87f)


Invalid value for VR IS: 'F'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
[2023-07-27 19:04:09,929] [INFO] (root) - Finding series for Selection named: CT Series
[2023-07-27 19:04:09,929] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2023-07-27 19:04:09,930] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2023-07-27 19:04:09,932] [INFO] (root) - On attribute: 'Modality' to match value: '(?i)CT'
[2023-07-27 19:04:09,934] [INFO] (root) -     Series attribute Modality value: CT
[2023-07-27 19:04:09,935] [INFO] (root) - Series attribute string value did not match. Try regEx.
[2023-07-27 19:04:09,936] [INFO] (root) - Selected Series, UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239


Done performing execution of operator AHLIDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 30, Operator ID: 11da4fe5-a42d-48c2-8bef-395649d568f1)
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 30, Operator ID: ab480e4f-bad9-42a4-9c39-5645af309090)
Done performing execution of operator DICOMSeriesToVolumeOperator

Going to initiate execution of operator MonaiBundleInferenceOperator
Executing operator MonaiBundleInferenceOperator (Process ID: 30, Operator ID: 75e71372-be89-4028-804d-8b5a4c3b196c)


monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.
monai.transforms.io.dictionary SaveImaged.__init__:resample: Current default value of argument `resample=True` has been deprecated since version 1.1. It will be changed to `resample=False` in version 1.3.


Done performing execution of operator MonaiBundleInferenceOperator

Going to initiate execution of operator DICOMSegmentationWriterOperator
Executing operator DICOMSegmentationWriterOperator (Process ID: 30, Operator ID: c209b024-b12c-487d-a7a4-670fbb9b1d83)


The string "C3N-00198" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
[2023-07-27 19:04:50,845] [INFO] (highdicom.seg.sop) - add plane #0 for segment #1
A value of type 'int64' cannot be assigned to a tag with VR UL.
A value of type 'int64' cannot be assigned to a tag with VR US.
[2023-07-27 19:04:50,847] [INFO] (highdicom.seg.sop) - add plane #1 for segment #1
[2023-07-27 19:04:50,850] [INFO] (highdicom.seg.sop) - add plane #2 for segment #1
[2023-07-27 19:04:50,852] [INFO] (highdicom.seg.sop) - add plane #3 for segment #1
[2023-07-27 19:04:50,857] [INFO] (hig

Done performing execution of operator DICOMSegmentationWriterOperator



In [10]:
import itk
from itkwidgets import view

outputimg = itk.imread('output/'+os.listdir('output')[0])

In [11]:
viewer = view(outputimg)

<IPython.core.display.Javascript object>

In [ ]:
viewer.set_image_gradient_opacity(0.4)